<div class="alert alert-block alert-success">
     
# IC PCDA CAPSTONE - HARRY BAKHSHI - NOTEBOOK 1 - DATA EXTRACTION AND CLEANING
     
</div>

In [1]:
import pandas as pd
import numpy as np
import pickle
import time
pd.options.mode.chained_assignment = None  # default='warn'

Data from: https://datahack.analyticsvidhya.com/contest/janatahack-demand-forecasting/True/#ProblemStatement

In [2]:
df = pd.read_csv('/Users/harrybakhshi/Desktop/Python_notes/data/train_0irEZ2H.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150150 entries, 0 to 150149
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   record_ID        150150 non-null  int64  
 1   week             150150 non-null  object 
 2   store_id         150150 non-null  int64  
 3   sku_id           150150 non-null  int64  
 4   total_price      150149 non-null  float64
 5   base_price       150150 non-null  float64
 6   is_featured_sku  150150 non-null  int64  
 7   is_display_sku   150150 non-null  int64  
 8   units_sold       150150 non-null  int64  
dtypes: float64(2), int64(6), object(1)
memory usage: 10.3+ MB
None


Null values in df col 4 - drop:

In [3]:
col_indices = [4]
for col in range(len(col_indices)):
    for i in df.index:
        val = df.loc[i, df.columns.values[col_indices[col]]]
        if pd.isnull(val):
            df.drop(i, inplace = True)
print(df.info()) #see results on dataframe

<class 'pandas.core.frame.DataFrame'>
Index: 150149 entries, 0 to 150149
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   record_ID        150149 non-null  int64  
 1   week             150149 non-null  object 
 2   store_id         150149 non-null  int64  
 3   sku_id           150149 non-null  int64  
 4   total_price      150149 non-null  float64
 5   base_price       150149 non-null  float64
 6   is_featured_sku  150149 non-null  int64  
 7   is_display_sku   150149 non-null  int64  
 8   units_sold       150149 non-null  int64  
dtypes: float64(2), int64(6), object(1)
memory usage: 15.5+ MB
None


Convert week column to datetime64[ns] format in both dfs:

In [4]:
print(df.columns.values[1])
# ensure timestamps in datetime64[ns] format
df[df.columns.values[1]] = pd.to_datetime(df[df.columns.values[1]], format='mixed')
#^https://saturncloud.io/blog/convert-pandas-column-to-datetime-a-guide/#:~:text=To%20convert%20a%20pandas%20column,new%20column%20with%20datetime%20values.
#Accessed 06/06/2024
import pandas.api.types as ptypes
print(ptypes.is_datetime64_dtype(df['week']))
#^https://stackoverflow.com/questions/25043620/correct-way-to-check-if-pandas-dataframe-index-is-a-certain-type-datetimeindex
#Accessed 06/06/2024
df = df.rename(columns={"week":"timestamp"})
#^https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html
#Accessed 06/06/2024
df = df.sort_values(by='timestamp')
#^https://blog.hubspot.com/website/pandas-sortby#:~:text=Pandas%20Sort%20by%20Column&text=Sorting%20data%20within%20a%20dataframe,the%20values%20in%20descending%20order.
#Accessed 05/06/2024

week
True


In [5]:
df.shape[0]

150149

Explore data:

In [6]:
df.head(10)

,record_ID,timestamp,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold
33494,47417,2011-01-08,9984,679023,180.2625,213.0375,0,0,21
32721,46310,2011-01-08,9112,216419,88.3500,119.7000,0,1,154
32722,46311,2011-01-08,9112,300021,86.2125,116.1375,0,1,128
32723,46314,2011-01-08,9112,216425,128.9625,128.9625,0,0,38
32724,46315,2011-01-08,9112,216233,133.9500,133.9500,0,0,89
32725,46316,2011-01-08,9112,217390,141.0750,166.7250,0,0,156
32726,46319,2011-01-08,9112,219844,288.5625,288.5625,0,0,28
32720,46309,2011-01-08,9112,216418,87.6375,119.7000,0,0,166
32727,46320,2011-01-08,9112,219009,227.2875,227.2875,0,0,75
32729,46322,2011-01-08,9112,222087,227.2875,227.2875,0,0,123


In [7]:
df.describe()

,record_ID,timestamp,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold
count,150149.000000,150149,150149.000000,150149.000000,150149.000000,150149.000000,150149.000000,150149.000000,150149.000000
mean,106270.971795,2012-04-18 17:50:55.213154816,9199.420935,254761.195226,206.626751,219.424262,0.095612,0.133201,51.674543
min,1.000000,2011-01-08 00:00:00,8023.000000,216233.000000,41.325000,61.275000,0.000000,0.000000,1.000000
25%,53111.000000,2011-08-22 00:00:00,8562.000000,217217.000000,130.387500,133.237500,0.000000,0.000000,20.000000
50%,106226.000000,2012-04-09 00:00:00,9371.000000,222087.000000,198.075000,205.912500,0.000000,0.000000,35.000000
75%,159452.000000,2012-11-20 00:00:00,9731.000000,245338.000000,233.700000,234.412500,0.000000,0.000000,62.000000
max,212644.000000,2013-12-03 00:00:00,9984.000000,679023.000000,562.162500,562.162500,1.000000,1.000000,2876.000000
std,61385.825580,NaN,615.593192,85547.587866,103.308516,110.960204,0.294059,0.339793,60.207962


Observation: data over 1061 days (2 years, 10 months, 26 days, or 34 months 26 days, or 151 weeks 4 days (including end date) from 8th Jan 2011 to 3rd Dec 2013     
^https://www.timeanddate.com/date/durationresult.html?d1=08&m1=01&y1=2011&d2=03&m2=12&y2=2013&ti=on

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150149 entries, 33494 to 129937
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   record_ID        150149 non-null  int64         
 1   timestamp        150149 non-null  datetime64[ns]
 2   store_id         150149 non-null  int64         
 3   sku_id           150149 non-null  int64         
 4   total_price      150149 non-null  float64       
 5   base_price       150149 non-null  float64       
 6   is_featured_sku  150149 non-null  int64         
 7   is_display_sku   150149 non-null  int64         
 8   units_sold       150149 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(6)
memory usage: 11.5 MB


In [9]:
print(ptypes.is_datetime64_dtype(df['timestamp']))
#^https://stackoverflow.com/questions/25043620/correct-way-to-check-if-pandas-dataframe-index-is-a-certain-type-datetimeindex
#Accessed 06/06/2024

True


In [10]:
df = df.sort_values(by='timestamp')
#^https://blog.hubspot.com/website/pandas-sortby#:~:text=Pandas%20Sort%20by%20Column&text=Sorting%20data%20within%20a%20dataframe,the%20values%20in%20descending%20order.
#Accessed 05/06/2024
df.head(1157)

,record_ID,timestamp,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold
33494,47417,2011-01-08,9984,679023,180.2625,213.0375,0,0,21
33459,47366,2011-01-08,9954,245338,469.5375,469.5375,0,0,9
33486,47407,2011-01-08,9984,222087,227.2875,227.2875,0,0,38
33492,47415,2011-01-08,9984,245338,469.5375,469.5375,0,0,10
33491,47412,2011-01-08,9984,223153,213.0375,213.0375,0,0,14
...,...,...,...,...,...,...,...,...,...
32345,45793,2011-01-08,8091,219009,227.2875,227.2875,0,0,7
32344,45789,2011-01-08,8091,217390,142.5000,168.8625,0,0,24
32350,45802,2011-01-08,8095,216419,89.0625,89.0625,0,0,92
3,4,2011-01-17,8091,216233,133.9500,133.9500,0,0,44


Data attributes:
    
* record_ID = Unique ID for each timestamp + product + and store combination recorded in the data (categorical, nominal)  
    
    An sku (stock keeping unit) is "a unique identifier for a product, typically assigned by a retailer or manufacturer", "used to track inventory" and "typically associated with a product's barcode", "the unique identifier for a specific product"

^https://www.shopify.com/uk/blog/what-is-a-stock-keeping-unit (Accessed 08/06/2024)
    
* timestamp = date of record (day in a given week when record made) (numeric, time series)       
* store_id = Unique ID for each store (no numerical order to be assumed) (categorical, nominal)            
* sku_id = Unique ID for each product (no numerical order to be assumed) (categorical, nominal)
* total_price = Sales Price of the product (numeric, float)                
* base_price = Base price of the product (numeric, float)

    The base price of a product is "the initial cost of a product or service before any additional fees or taxes are added" such as delivery, registrations required, taxes applying to the product/service. It may change to 'maintain profitability' if the production costs, supply and demand, competition, and market trends change. (1)
  
  Profitability is the property of something that it produces or is likely to produce profit. (2)    
  "profitability - the fact that something produces or is likely to produce a profit" (2)    
^(1) https://fastercapital.com/topics/what-is-base-price.html (Accessed 08/06/2024)  
^(2) https://dictionary.cambridge.org/dictionary/english/profitability (Accessed 08/06/2024)
    
* is_featured_sku = unique item was part of the featured item of the week (categorical, binary)
* is_display_sku = unique item was on display at a prominent place at the store (categorical, binary)
* units_sold = Total Units sold for the sku, in the given week of the timestamp and/or between the last time stamp and the current timestamp, and the recorded store of sale (numeric, int)

^https://datahack.analyticsvidhya.com/contest/janatahack-demand-forecasting/True/#ProblemStatement
(Accessed 07/06/2024)
    
       

Plan:   
Do multivariate time series with variables:
    
    timestamp:
    timestamp = date of record (day in a given week when record made) (numeric, time series) 
    
    Variables:
    total_price = Sales Price of the product (numeric, float)                
    base_price = Base price of the product (numeric, float)
    units_sold = Total Units sold for the sku, in the given week of the timestamp and/or between the last time stamp and the current timestamp, and the recorded store of sale (numeric, int)
    is_featured_sku = unique item was part of the featured item of the week (categorical, binary)
    is_display_sku = unique item was on display at a prominent place at the store (categorical, binary)
                
    sku_id = Unique ID for each product (no numerical order to be assumed) (categorical, nominal)
    
    Create new variables (percent_featured, percent_display); these and total_price, base_price, units_sold for each sku_id (= product) to take a value at each timestamp

      
Forecast the product sales price, base price, units sold, for a given timestamp where the time is a date in the sales period (from week to week between timestamps) by which the recorded sales were made 
    
Able to work out at a specific time of interest, what product is selling in what volume
    
Useful for predicting how products will sell and for what sales and base prices (what sales + base prices sell more/less of a product)

Drop cols not needed:

In [11]:
#Drop cols not needed:
col_indices = [0]
for col in range(len(col_indices)):
    df = df.drop(df.columns.values[col_indices[col]], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150149 entries, 33494 to 129937
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   timestamp        150149 non-null  datetime64[ns]
 1   store_id         150149 non-null  int64         
 2   sku_id           150149 non-null  int64         
 3   total_price      150149 non-null  float64       
 4   base_price       150149 non-null  float64       
 5   is_featured_sku  150149 non-null  int64         
 6   is_display_sku   150149 non-null  int64         
 7   units_sold       150149 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(5)
memory usage: 10.3 MB


In [12]:
df.index = np.arange(0, len(df))
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150149 entries, 0 to 150148
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   timestamp        150149 non-null  datetime64[ns]
 1   store_id         150149 non-null  int64         
 2   sku_id           150149 non-null  int64         
 3   total_price      150149 non-null  float64       
 4   base_price       150149 non-null  float64       
 5   is_featured_sku  150149 non-null  int64         
 6   is_display_sku   150149 non-null  int64         
 7   units_sold       150149 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(5)
memory usage: 10.3 MB


In [13]:
df.head()

,timestamp,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold
0,2011-01-08,9984,679023,180.2625,213.0375,0,0,21
1,2011-01-08,9954,245338,469.5375,469.5375,0,0,9
2,2011-01-08,9984,222087,227.2875,227.2875,0,0,38
3,2011-01-08,9984,245338,469.5375,469.5375,0,0,10
4,2011-01-08,9984,223153,213.0375,213.0375,0,0,14


How many stores and products?

In [14]:
col_indices = [1, 2]
lists_unique_entries = []
for col in range(len(col_indices)):
    entry_list = df[df.columns.values[col_indices[col]]]
    # #How many elements?
    # print('Elem after:' + ' ' + str(len(entry_list)))
    # print('')
    #Find unique members of the list:
    found_list = []
    unique_list = []
    entry = []
    entry_list_length = len(entry_list)
    for i in range(entry_list_length):
        entry = entry_list[i]
        if entry not in found_list:
            unique_list.append(entry)
            found_list.append(entry)
    lists_unique_entries.append(unique_list)
print(lists_unique_entries[0])
print('Stores: ', len(lists_unique_entries[0]))
print(lists_unique_entries[1])
print('Products: ', len(lists_unique_entries[1]))

[9984, 9954, 9961, 9890, 9909, 9879, 9881, 9880, 9731, 9876, 9872, 9809, 9845, 9823, 9837, 9745, 9789, 9813, 9770, 9456, 9713, 9680, 9611, 9700, 9613, 9632, 9532, 9672, 9578, 9498, 9479, 9490, 9481, 9436, 9432, 9430, 9439, 9425, 9371, 9442, 8091, 9281, 9328, 9273, 9279, 9190, 9250, 9221, 8869, 9164, 9178, 9147, 9132, 9112, 9092, 9043, 8991, 9001, 8911, 8562, 8400, 8438, 8555, 8422, 8317, 8398, 8319, 8392, 8218, 8121, 8222, 8023, 8058, 8095, 8063, 8094]
Stores:  76
[679023, 245338, 222087, 223153, 223245, 222765, 547934, 378934, 219029, 216418, 219009, 217390, 216233, 216425, 320485, 300021, 245387, 216419, 398721, 217217, 217777, 219844, 600934, 673209, 327492, 300291, 546789, 545621]
Products:  28


Multiple product sales, sales prices, base prices recorded in time series for each timestamp - panel data:     
Bin the time series for all units, ignoring any temporal ordering within the bin input values, with equal bin size in timestamps/time for all units (1); use median over mean (median more robust to outliers if bin data distribution not symmetric (2); if bin data distribution symmetric, mean = median, so would be using median (3) if use mean in this case - instead use median for all)    
^(1) - https://stats.stackexchange.com/questions/129900/machine-learning-algorithms-for-panel-data        
Accessed 09/06/2024    
^(2) - Statistics: Unlocking the Power of Data (Robin H. Lock, Patti Frazer Lock, Kari Lock Morgan, Eric F. Lock, Dennis F. Lock, 2021)   
Accessed 08/12/2023    
^(3) - https://online.stat.psu.edu/stat200/lesson/2/2.2/2.2.4/2.2.4.1    
Accessed 13/06/2024   

For EDA consider median total_price, base_price, units_sold, and measures percent_featured and percent_display, for all products in all stores for each timestamp:   

In [138]:
from warnings import simplefilter
import statistics
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
#^https://stackoverflow.com/questions/68292862/performancewarning-dataframe-is-highly-fragmented-this-is-usually-the-result-o
#Accessed 08/06/2024)

#Get unique timestamps:
df_col = df[df.columns.values[0]]
type_list = []
for i in df_col.index:
    entry = df_col[i]
    try:
        found_type = str(entry)
        type_list.append(entry)
    except:
        pass
# #How many elements?
# print('Elem after:' + ' ' + str(len(type_list)))
# print('')
#Find unique members of the list:
found_list = []
unique_in_type_list = []
entry = []
type_list_length = len(type_list)
for i in range(type_list_length):
    entry = type_list[i]
    if entry not in found_list:
        unique_in_type_list.append(entry)
        found_list.append(entry)
        
#Make EDA df:
d = {}
col_list = ['timestamp','total_price', 'base_price', 'units_sold']
col = 0
while col == 0:
    d[(col_list[col])] = ([float(0)] * len(unique_in_type_list))
    col += 1
col = 1
while col < 4:
    d[('median_'+ col_list[col])] = ([float(0)] * len(unique_in_type_list))
    col += 1
col = 4
while col < 5:
    d['percent_featured'] = ([float(0)] * len(unique_in_type_list))
    col += 1
col = 5
while col < 6:
    d['percent_display'] = ([float(0)] * len(unique_in_type_list))
    col += 1
eda_ts_df = pd.DataFrame(data=d)
eda_ts_df['timestamp'] = pd.to_datetime(eda_ts_df['timestamp'], format='mixed')
#^https://saturncloud.io/blog/convert-pandas-column-to-datetime-a-guide/#:~:text=To%20convert%20a%20pandas%20column,new%20column%20with%20datetime%20values.
#Accessed 06/06/2024
for timestamp in range(len(unique_in_type_list)):
    entry = unique_in_type_list[timestamp]
    eda_ts_df[eda_ts_df.columns.values[0]][timestamp] = entry
eda_ts_df['timestamp'] = pd.to_datetime(eda_ts_df['timestamp'], format='mixed')
#^https://saturncloud.io/blog/convert-pandas-column-to-datetime-a-guide/#:~:text=To%20convert%20a%20pandas%20column,new%20column%20with%20datetime%20values.
#Accessed 06/06/2024
eda_ts_df = eda_ts_df.sort_values(by='timestamp')
#^https://blog.hubspot.com/website/pandas-sortby#:~:text=Pandas%20Sort%20by%20Column&text=Sorting%20data%20within%20a%20dataframe,the%20values%20in%20descending%20order.
#Accessed 05/06/2024
print(ptypes.is_datetime64_dtype(eda_ts_df['timestamp']))
#^https://stackoverflow.com/questions/25043620/correct-way-to-check-if-pandas-dataframe-index-is-a-certain-type-datetimeindex
#Accessed 06/06/2024
start = time.time()
print('Start!')
for timestamp in range(len(unique_in_type_list)):
    sales_price_list_per_ts = []
    base_price_list_per_ts = []
    units_sold_list_per_ts = []
    is_featured_list_per_ts = []
    is_display_list_per_ts = []
    for row in df.index:
        if df['timestamp'][row] == eda_ts_df['timestamp'][timestamp]:
            sales_price_list_per_ts.append(df['total_price'][row])
            base_price_list_per_ts.append(df['base_price'][row])
            units_sold_list_per_ts.append(df['units_sold'][row])
            is_featured_list_per_ts.append(df['is_featured_sku'][row])
            is_display_list_per_ts.append(df['is_display_sku'][row])
    median_sales_price_per_ts = statistics.median(sales_price_list_per_ts)
    median_base_price_per_ts = statistics.median(base_price_list_per_ts)
    median_units_sold_per_ts = statistics.median(units_sold_list_per_ts)
    
    #can't use mode or median as instances of is_featured_sku or is_display_sku can be too 
    #infrequent to detect the existing featuring or displaying in this way in binning multiple retailings of an 
    #individual product in all
    #the different stores per timestamp - use mean; as is_featured count / all product observations per timestamp,
    #is_display count / all product observations per timestamp
    #Measure of how much featuring a product/displaying it was emphasised at that timestamp - for EDA
    #considering all products in all stores per timestamp so measure of how much featuring 
    #products/displaying them was emphasised at that timestamp for all the products considered in all
    #the stores considered

    #Use this rather than sum because describes no of values in bin as well as no featured, whereas sum does not; 
    #as mean ok because keep outliers in potentially skewed bin input value distributions (which as discrete 
    #distributions can of course be skewed and have a mean) by Grubbs 1969 definition of an outlier (1), 
    #because after considering the cause of the outliers and obviously eliminating errors, if trying to model a 
    #population that includes these outliers, it is important to include them, as they are part of the population (2)
    #(1) - Module 12, IC PCDA: ‘Required discussion 12.3: Anomaly detection resources’ - Harry Bakhshi. Accessed 
    #18/05/2024.
    #(2) - Module 12, IC PCDA: 'Mini-lesson 12.2: Consequences of removing outliers'. Accessed 09/06/2024
    
    percent_featured_per_ts = sum(is_featured_list_per_ts) / len(is_featured_list_per_ts)
    percent_display_per_ts = sum(is_display_list_per_ts) / len(is_display_list_per_ts) 
    eda_ts_df['median_total_price'][timestamp] = median_sales_price_per_ts
    eda_ts_df['median_base_price'][timestamp] = median_base_price_per_ts
    eda_ts_df['median_units_sold'][timestamp] = median_units_sold_per_ts
    eda_ts_df['percent_featured'][timestamp] = percent_featured_per_ts
    eda_ts_df['percent_display'][timestamp] = percent_display_per_ts
end = time.time()
print('secs: ', end - start) # time in seconds
print('min: ', (end - start)/60) # time in min
print(eda_ts_df.head())

True
Start!
secs:  134.64049577713013
min:  2.244008262952169
   timestamp  median_total_price  median_base_price  median_units_sold  \
0 2011-01-08            205.9125           205.9125               28.0   
1 2011-01-17            190.2375           205.9125               38.0   
2 2011-01-24            190.2375           205.9125               37.0   
3 2011-01-31            210.1875           210.9000               33.0   
4 2011-02-05            210.9000           211.6125               31.0   

   percent_featured  percent_display  
0          0.066667         0.077922  
1          0.189610         0.163636  
2          0.274459         0.195671  
3          0.010390         0.096104  
4          0.000000         0.089177  


In [162]:
eda_ts_df.head()

,timestamp,median_total_price,median_base_price,median_units_sold,percent_featured,percent_display
0,2011-01-08,205.9125,205.9125,28.0,0.066667,0.077922
1,2011-01-17,190.2375,205.9125,38.0,0.189610,0.163636
2,2011-01-24,190.2375,205.9125,37.0,0.274459,0.195671
3,2011-01-31,210.1875,210.9000,33.0,0.010390,0.096104
4,2011-02-05,210.9000,211.6125,31.0,0.000000,0.089177


Bin data for time series - 1 value for each product variable per timestamp:

In [140]:
ts_df = eda_ts_df.copy()
col_list = ['timestamp','total_price', 'base_price', 'units_sold', 'percent_featured', 'percent_display']
col_list_indices = [1, 2, 3, 4, 5]
col = 0
while col < 3:
    for product in range(len(lists_unique_entries[1])):
        entry = str(lists_unique_entries[1][product]) + ('median_' + col_list[col_list_indices[col]])
        ts_df.loc[:, entry] = pd.Series(np.zeros(len(unique_in_type_list)), index=ts_df.index)
    col += 1
col = 3
while col < 4:
    for product in range(len(lists_unique_entries[1])):
        entry = str(lists_unique_entries[1][product]) + ('percent_featured')
        ts_df.loc[:, entry] = pd.Series(np.zeros(len(unique_in_type_list)), index=ts_df.index)
    col += 1
while col < 5:
    for product in range(len(lists_unique_entries[1])):
        entry = str(lists_unique_entries[1][product]) + ('percent_display')
        ts_df.loc[:, entry] = pd.Series(np.zeros(len(unique_in_type_list)), index=ts_df.index)
    col += 1
drop_list = ['median_total_price', 'median_base_price', 'median_units_sold', 'percent_featured', 'percent_display']
ts_df = ts_df.drop(drop_list, axis=1)
# ts_df.head()
col_list = ['total_price', 'base_price', 'units_sold', 'is_featured_sku', 'is_display_sku']
col_list_2 = ['total_price', 'base_price', 'units_sold', 'percent_featured', 'percent_display']
start = time.time()
print('Start!')
for product in range(len(lists_unique_entries[1])):
    # for timestamp in range(1):
    for timestamp in range(len(unique_in_type_list)):
        for col in range(len(col_list)):
            #Create lists for central tendency measures - see lower code
            sales_price_list_per_ts_per_product = []
            base_price_list_per_ts_per_product = []
            units_sold_list_per_ts_per_product = []
            is_featured_list_per_ts_per_product = []
            is_display_list_per_ts_per_product = []
            for row in df.index:
                if df['timestamp'][row] == ts_df['timestamp'][timestamp]:
                    if df['sku_id'][row] == int(str(lists_unique_entries[1][product])):
                        #Append to lists for for central tendency measures - see lower code
                        if col_list[col] == 'total_price':
                            sales_price_list_per_ts_per_product.append(df[col_list[col]][row])
                        if col_list[col] == 'base_price':
                            base_price_list_per_ts_per_product.append(df[col_list[col]][row])
                        if col_list[col] == 'units_sold':
                            units_sold_list_per_ts_per_product.append(df[col_list[col]][row])
                        if col_list[col] == 'is_featured_sku':
                            is_featured_list_per_ts_per_product.append(df[col_list[col]][row])
                        if col_list[col] == 'is_display_sku':
                            is_display_list_per_ts_per_product.append(df[col_list[col]][row])
            #In case sales of the same product in different stores at the timestamp take the median for the product at that timestamp:
            if col_list[col] == 'total_price':
                ts_df[(str(lists_unique_entries[1][product]) + ('median_' + col_list[col]))][timestamp] = statistics.median(sales_price_list_per_ts_per_product)  
            if col_list[col] == 'base_price':
                ts_df[(str(lists_unique_entries[1][product]) + ('median_' + col_list[col]))][timestamp] = statistics.median(base_price_list_per_ts_per_product)  
            if col_list[col] == 'units_sold':
                ts_df[(str(lists_unique_entries[1][product]) + ('median_' + col_list[col]))][timestamp] = statistics.median(units_sold_list_per_ts_per_product)  
            
            #can't use mode or median as instances of is_featured_sku or is_display_sku can be too 
            #infrequent to detect the existing featuring or displaying in this way in binning multiple retailings of an individual 
            #product in all
            #the different stores per timestamp - use mean; as is_featured count / all product observations per timestamp,
            #is_display count / all product observations per timestamp
            #Measure of how much featuring a product/displaying it was emphasised at that timestamp

            #Use this rather than sum because describes no of values in bin as well as no featured, whereas sum does not; 
            #as mean ok because keep outliers in potentially skewed bin input value distributions (which as discrete 
            #distributions can of course be skewed and have a mean) by Grubbs 1969 definition of an outlier (1), 
            #because after considering the cause of the outliers and obviously eliminating errors, if trying to model a 
            #population that includes these outliers, it is important to include them, as they are part of the population (2)
            #(1) - Module 12, IC PCDA: ‘Required discussion 12.3: Anomaly detection resources’ - Harry Bakhshi. Accessed 
            #18/05/2024.
            #(2) - Module 12, IC PCDA: 'Mini-lesson 12.2: Consequences of removing outliers'. Accessed 09/06/2024
            
            if col_list_2[col] == 'percent_featured':
                ts_df[(str(lists_unique_entries[1][product]) + ('percent_featured'))][timestamp] = sum(is_featured_list_per_ts_per_product) / len(is_featured_list_per_ts_per_product)  
            if col_list_2[col] == 'percent_display':
                ts_df[(str(lists_unique_entries[1][product]) + ('percent_display'))][timestamp] = sum(is_display_list_per_ts_per_product) / len(is_display_list_per_ts_per_product)
end = time.time()
print('secs: ', end - start) # time in seconds
print('min: ', (end - start)/60) # time in min

Start!
secs:  17648.841105222702
min:  294.1473517537117


In [159]:
#Started at --:--, takes approx. 5hrs 10 min (2.38 min per timestamp, 130 ts's), Expect done at
#--:-- - took 295 mins last time

#### This is how you would do it for making a time series variable per store per product (10640 variables with 130 timestamp values, using parallel computing, with ability to run on either Jupyter or e.g. Spyder (offline) and ability to pause/resume/quit execution). Did it to show possible

- would find seasonality, find and treat outliers, then check forecastability and drop variables not forecastable (using same code as for notebooks 4, 5, 6, 7)
- For this data:   
    Started at 17:05 21st Jun, expect take approx. 72.6 hrs (3.03 days) (33.52 min per timestamp)  
    Expected done at 17:43 on Monday 24nd Jun   
    Finished at 11:16 Mon 1 Jul   
    secs:  843078.8388700485   
    min:  14051.313981167476   
    time for 130 timestamps in hrs:  234.18856635279127   
    time for 130 timestamps in days:  9.757856931366302   

In [ ]:
# from janatahack_panel_data_defs import panel_to_ts_janatahack
# #^https://stackoverflow.com/questions/41385708/multiprocessing-example-giving-attributeerror
# #Accessed 15/06/2024
# from warnings import simplefilter
# simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
# #^https://stackoverflow.com/questions/68292862/performancewarning-dataframe-is-highly-fragmented-this-is-usually-the-result-o
# #Accessed 08/06/2024)

# file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/janatahack_demand_forecasting_data_eda_ts.pik'
# # with open(file, 'wb') as f:
# #     pickle.dump(eda_ts_df, f) #write df to .pik file on disk
# with open(file, 'rb') as f:
#      eda_ts_df = pickle.load(f) #load pickle file 'file' into variable
# file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/janatahack_demand_forecasting_lists_unique_entries.pik'
# # with open(file, 'wb') as f:
# #     pickle.dump(lists_unique_entries, f) #write df to .pik file on disk
# with open(file, 'rb') as f:
#      lists_unique_entries = pickle.load(f) #load pickle file 'file' into variable
    
# ts_df = eda_ts_df.copy()
# col_list = ['timestamp','total_price', 'base_price', 'units_sold', 'percent_featured', 'percent_display']
# col_list_indices = [1, 2, 3, 4, 5]
# col = 0
# store_product_list = []
# while col < 3:
#     for store in range(len(lists_unique_entries[0])):    
#         for product in range(len(lists_unique_entries[1])):
#             entry = str(lists_unique_entries[0][store]) + '_' + str(lists_unique_entries[1][product]) + ('median_' + col_list[col_list_indices[col]])
#             store_product_list.append(entry)
#             ts_df.loc[:, entry] = pd.Series(np.zeros(len(ts_df)), index=ts_df.index)
#     col += 1
# col = 3
# while col < 4:
#     for store in range(len(lists_unique_entries[0])):  
#         for product in range(len(lists_unique_entries[1])):
#             entry = str(lists_unique_entries[0][store]) + '_' + str(lists_unique_entries[1][product]) + ('percent_featured')
#             store_product_list.append(entry)
#             ts_df.loc[:, entry] = pd.Series(np.zeros(len(ts_df)), index=ts_df.index)
#     col += 1
# while col < 5:
#     for store in range(len(lists_unique_entries[0])):  
#         for product in range(len(lists_unique_entries[1])):
#             entry = str(lists_unique_entries[0][store]) + '_' + str(lists_unique_entries[1][product]) + ('percent_display')
#             store_product_list.append(entry)
#             ts_df.loc[:, entry] = pd.Series(np.zeros(len(ts_df)), index=ts_df.index)
#     col += 1
# drop_list = ['median_total_price', 'median_base_price', 'median_units_sold', 'percent_featured', 'percent_display']
# ts_df = ts_df.drop(drop_list, axis=1)
# col_list = ['total_price', 'base_price', 'units_sold', 'is_featured_sku', 'is_display_sku']
# col_list_2 = ['total_price', 'base_price', 'units_sold', 'percent_featured', 'percent_display']

# file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/janatahack_demand_forecasting_store_product_list.pik'
# with open(file, 'wb') as f:
#     pickle.dump(store_product_list, f) #write df to .pik file on disk
# # with open(file, 'rb') as f:
# #      store_product_list = pickle.load(f) #load pickle file 'file' into variable
# _ts_df = ts_df
# file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/janatahack_demand_forecasting_data__ts_df.pik'
# with open(file, 'wb') as f:
#     pickle.dump(_ts_df, f) #write df to .pik file on disk
# # with open(file, 'rb') as f:
# #      _ts_df = pickle.load(f) #load pickle file 'file' into variable
# file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/janatahack_demand_forecasting_data_df.pik'
# # with open(file, 'wb') as f:
# #     pickle.dump(df, f) #write df to .pik file on disk
# with open(file, 'rb') as f:
#       df = pickle.load(f) #load pickle file 'file' into variable

# import multiprocessing 
# # from multiprocessing import set_start_method
# # set_start_method("spawn")
# from multiprocessing import Pool
# #^https://python.omics.wiki/multiprocessing_map/multiprocessing_partial_function_multiple_arguments
# #^https://www.sitepoint.com/python-multiprocessing-parallel-programming/
# #Both accessed 15/06/2024

# # sec_list = []

# file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/janatahack_demand_forecasting_data__ts_df.pik'
# # with open(file, 'wb') as f:
# #     pickle.dump(ts_df, f) #write df to .pik file on disk
# with open(file, 'rb') as f:
#       ts_df = pickle.load(f) #load pickle file 'file' into variable
      
# print('Expected time in hrs: ', ((1.5121979713439941/8)*10640*130)/(60*60))
# print('Expected time in days: ', ((1.5121979713439941/8)*10640*130)/(60*60*24))
# size = int(len(store_product_list))
# # size = int(16)
# chunksize = int(8)
# task_denom = 8 #no of simultaneous processes to do safely possible, can't be greater than size
# result = math.floor(size / task_denom)
# #^https://www.w3schools.com/python/ref_math_floor.asp#:~:text=The%20math.,necessary%2C%20and%20returns%20the%20result.
# #Accessed 06/07/2024
# remainder = size % task_denom
# #^https://runestone.academy/ns/books/published/thinkcspy/SimplePythonData/OperatorsandOperands.html#
# #Accessed 06/07/2024
# tasks = result + math.ceil(remainder/task_denom)
# print(tasks)
# # chunksize = int(size/8)
# print('Start!')
# if __name__ == "__main__":
#     with Pool(processes=8) as pool: #8 cores on my laptop
#         start1 = time.time()
#         step = 0
#         step_count = 0
#         ts_df_temp = ts_df.drop(ts_df.columns.values[1:], axis=1)
#         while step_count < tasks:
#             if step_count == tasks:
#                 break
#             #^https://www.geeksforgeeks.org/how-to-use-while-true-in-python/
#             #Accessed 20/06/2024
#             for ele in range(int(len(list(range(len(store_product_list)))[:size])/task_denom)):
#                 try:
#                     map_list = list(pool.map(panel_to_ts_janatahack, list(range(len(store_product_list)))[(ele + step):(ele + step + task_denom)], chunksize=chunksize))        
#                     #^https://python.omics.wiki/multiprocessing_map/multiprocessing_partial_function_multiple_arguments
#                     #^https://www.sitepoint.com/python-multiprocessing-parallel-programming/
#                     #Both accessed 15/06/2024
#                     #^https://stackoverflow.com/questions/1303347/getting-a-map-to-return-a-list-in-python-3-x
#                     #^https://stackoverflow.com/questions/58841709/how-to-store-multiple-pandas-dataframes-together
#                     #^https://python.omics.wiki/multiprocessing_map/multiprocessing_partial_function_multiple_arguments
#                     #^https://docs.python.org/3/library/multiprocessing.html#multiprocessing.pool.Pool
#                     #All accessed 15/06/2024
#                     # start2 = time.time()
#                     for item in range(len(map_list)):
#                         ts_df_temp_2 = map_list[item]
#                         ts_df_temp = pd.merge(
#                             ts_df_temp, 
#                             ts_df_temp_2, 
#                             left_index=True, 
#                             right_index=True
#                         )
#                     step = step + (task_denom - 1)
#                     step_count += 1
#                     # end2 = time.time()
#                 except KeyboardInterrupt:
#                     try:
#                         print('\nPausing...  (Hit ENTER to continue, type quit to exit.)')
#                         response = input()
#                         if response == 'quit':
#                             import sys
#                             sys.exit()
#                         else:
#                             print('Resuming...')
#                     except KeyboardInterrupt:
#                         print('Resuming...')
#                         continue
#                 #^https://stackoverflow.com/questions/62779180/pandas-merge-unexpectedly-produces-suffixes
#                 #Accessed 15/06/2024
#                 #^https://stackoverflow.com/questions/7180914/pause-resume-a-python-script-in-middle
#                 #^https://stackoverflow.com/questions/73663/how-do-i-terminate-a-script
#                 #Accessed 20/06/2024
#         end1 = time.time()
#         # sec_list.append((end1 - start1))
#         print('secs: ', (end1 - start1)) # time in seconds
#         print('min: ', (end1 - start1)/60) # time in min
#         print('time for 130 timestamps in hrs: ', (end1 - start1)/(60*60)) # time in hrs
#         print('time for 130 timestamps in days: ', (end1 - start1)/(60*60*24)) # time in days       
#     # exiting the 'with'-block has stopped the pool
#     print("Now the pool is closed and no longer available")
#     pool.terminate()
# ts_df = ts_df_temp
# print(ts_df['9984_679023median_total_price'][100:])
# print(ts_df['9984_245338median_total_price'][0:10])
# #Pickle multivariate timeseries:
# file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/janatahack_demand_forecasting_data_ts_df_parallel.pik'
# # with open(file, 'wb') as f:
# #     pickle.dump(ts_df, f) #write df to .pik file on disk
# with open(file, 'rb') as f:
#       ts_df = pickle.load(f) #load pickle file 'file' into variable
# file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/janatahack_demand_forecasting_data_ts_df_parallel_2.pik'
# # with open(file, 'wb') as f:
# #     pickle.dump(ts_df, f) #write df to .pik file on disk
# with open(file, 'rb') as f:
#       ts_df = pickle.load(f) #load pickle file 'file' into variable
# print(ts_df.info())

In [22]:
#Pickle multivariate timeseries:
file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/janatahack_demand_forecasting_data_ts_df_parallel.pik'
# with open(file, 'wb') as f:
#     pickle.dump(ts_df_2, f) #write df to .pik file on disk
with open(file, 'rb') as f:
     ts_df_2 = pickle.load(f) #load pickle file 'file' into variable
file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/janatahack_demand_forecasting_data_ts_df_parallel_2.pik'
# with open(file, 'wb') as f:
#     pickle.dump(ts_df_2, f) #write df to .pik file on disk
with open(file, 'rb') as f:
      ts_df_2 = pickle.load(f) #load pickle file 'file' into variable

Test conversion:

In [5]:
df_2 = df.copy()[df.copy()['sku_id'] == 679023][df.copy()['store_id'] == 9984]
df_2.index = np.arange(0, len(df_2))
df_2.head(50)
import statistics

/tmp/ipykernel_18955/2221620203.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_2 = df.copy()[df.copy()['sku_id'] == 679023][df.copy()['store_id'] == 9984]


In [6]:
median_test_list = [] 
for row in range(1):
    median_test_list.append(df_2['total_price'][row])
_9984_679023median_total_price_ts_1 = statistics.median(median_test_list)  
_9984_679023median_total_price_ts_1

180.2625

In [7]:
median_test_list = [] 
for row in range(1):
    median_test_list.append(df_2['base_price'][row])
_9984_679023median_base_price_ts_1 = statistics.median(median_test_list) 
_9984_679023median_base_price_ts_1

213.0375

In [8]:
median_test_list = [] 
for row in range(1):
    median_test_list.append(df_2['units_sold'][row])
_9984_679023median_units_sold_ts_1 = statistics.median(median_test_list)
_9984_679023median_units_sold_ts_1

21

In [9]:
mean_test_list = [] 
for row in range(1):
    mean_test_list.append(df_2['is_featured_sku'][row])
_9984_679023percent_featured_ts_1 = sum(mean_test_list) / len(mean_test_list)
_9984_679023percent_featured_ts_1

0.0

In [10]:
mean_test_list = [] 
for row in range(1):
    mean_test_list.append(df_2['is_display_sku'][row])
_9984_679023percent_display_ts_1 = sum(mean_test_list) / len(mean_test_list)
_9984_679023percent_display_ts_1

0.0

In [23]:
ts_df_2['9984_679023median_base_price'][0]

213.0375

In [12]:
ts_df_2['9984_679023median_units_sold'][0]

21.0

In [13]:
ts_df_2['9984_679023percent_featured'][0]

0.0

In [14]:
ts_df_2['9984_679023percent_display'][0]

0.0

In [165]:
ts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Columns: 141 entries, timestamp to 545621percent_display
dtypes: datetime64[ns](1), float64(140)
memory usage: 143.3 KB


In [167]:
ts_df.head(2)

,timestamp,679023median_total_price,245338median_total_price,222087median_total_price,223153median_total_price,223245median_total_price,222765median_total_price,547934median_total_price,378934median_total_price,219029median_total_price,...,398721percent_display,217217percent_display,217777percent_display,219844percent_display,600934percent_display,673209percent_display,327492percent_display,300291percent_display,546789percent_display,545621percent_display
0,2011-01-08,180.2625,469.5375,226.93125,213.0375,205.2000,195.225,177.4125,205.9125,312.7875,...,0.055556,0.076923,0.10,0.107143,0.285714,0.5,0.0,0.222222,0.571429,0.0
1,2011-01-17,178.1250,426.7875,196.65000,190.2375,205.9125,240.825,177.4125,177.4125,312.0750,...,0.444444,0.230769,0.15,0.321429,0.000000,0.0,0.0,0.000000,0.000000,0.0


In [144]:
df_2 = df.copy()[df.copy()['sku_id'] == 679023]
df_2.index = np.arange(0, len(df_2))
df_2.head(50)

,timestamp,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold
0,2011-01-08,9984,679023,180.2625,213.0375,0,0,21
1,2011-01-08,9425,679023,180.9750,213.0375,1,1,17
2,2011-01-08,9430,679023,193.0875,213.0375,0,1,28
3,2011-01-08,9281,679023,186.6750,198.0750,0,1,5
4,2011-01-08,9092,679023,177.4125,213.0375,1,1,26
5,2011-01-08,8869,679023,191.6625,213.0375,0,1,38
6,2011-01-08,8317,679023,178.8375,213.0375,1,0,31
7,2011-01-08,8063,679023,177.4125,213.0375,1,0,5
8,2011-01-08,8094,679023,177.4125,213.0375,0,1,25
9,2011-01-17,8063,679023,183.1125,183.1125,0,0,8


In [168]:
median_test_list = [] 
for row in range(9):
    median_test_list.append(df_2['total_price'][row])
_679023median_total_price_ts_1 = statistics.median(median_test_list)  
_679023median_total_price_ts_1

180.2625

In [169]:
median_test_list = [] 
for row in range(9):
    median_test_list.append(df_2['base_price'][row])
_679023median_base_price_ts_1 = statistics.median(median_test_list) 
_679023median_base_price_ts_1

213.0375

In [170]:
median_test_list = [] 
for row in range(9):
    median_test_list.append(df_2['units_sold'][row])
_679023median_units_sold_ts_1 = statistics.median(median_test_list)
_679023median_units_sold_ts_1

25

In [171]:
mean_test_list = [] 
for row in range(9):
    mean_test_list.append(df_2['is_featured_sku'][row])
_679023percent_featured_ts_1 = sum(mean_test_list) / len(mean_test_list)
_679023percent_featured_ts_1

0.4444444444444444

In [172]:
mean_test_list = [] 
for row in range(9):
    mean_test_list.append(df_2['is_display_sku'][row])
_679023percent_display_ts_1 = sum(mean_test_list) / len(mean_test_list)
_679023percent_display_ts_1

0.6666666666666666

In [173]:
ts_df['679023median_base_price'][0]

213.0375

In [174]:
ts_df['679023median_units_sold'][0]

25.0

In [175]:
ts_df['679023percent_featured'][0]

0.4444444444444444

In [176]:
ts_df['679023percent_display'][0]

0.6666666666666666

In [177]:
ts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Columns: 141 entries, timestamp to 545621percent_display
dtypes: datetime64[ns](1), float64(140)
memory usage: 143.3 KB


Pickle multivariate timeseries:

In [178]:
#Pickle multivariate timeseries:
file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/janatahack_demand_forecasting_data_ts_df.pik'
with open(file, 'wb') as f:
    pickle.dump(ts_df, f) #write df to .pik file on disk
# with open(file, 'rb') as f:
#      ts_df = pickle.load(f) #load pickle file 'file' into variable

In [179]:
#Pickle multivariate timeseries:
file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/janatahack_demand_forecasting_data_ts_df_2.pik'
with open(file, 'wb') as f:
    pickle.dump(ts_df, f) #write df to .pik file on disk
# with open(file, 'rb') as f:
#      ts_df = pickle.load(f) #load pickle file 'file' into variable

In [156]:
eda_ts_df.head()

,timestamp,median_total_price,median_base_price,median_units_sold,percent_featured,percent_display
0,2011-01-08,205.9125,205.9125,28.0,0.066667,0.077922
1,2011-01-17,190.2375,205.9125,38.0,0.189610,0.163636
2,2011-01-24,190.2375,205.9125,37.0,0.274459,0.195671
3,2011-01-31,210.1875,210.9000,33.0,0.010390,0.096104
4,2011-02-05,210.9000,211.6125,31.0,0.000000,0.089177


In [163]:
file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/janatahack_demand_forecasting_data_eda_ts.pik'
with open(file, 'wb') as f:
    pickle.dump(eda_ts_df, f) #write df to .pik file on disk
# with open(file, 'rb') as f:
#      eda_ts_df = pickle.load(f) #load pickle file 'file' into variable

In [4]:
file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/janatahack_demand_forecasting_data_df.pik'
with open(file, 'wb') as f:
    pickle.dump(df, f) #write df to .pik file on disk
# with open(file, 'rb') as f:
#      df = pickle.load(f) #load pickle file 'file' into variable